Realizando imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve
import json

Carregando arquivos com os resultados das consultas

In [ ]:
try:
   with open('map_qrels.json', 'r', encoding='utf-8') as file:
       map_qrels = json.load(file)
   with open('map_results_mysql.json', 'r', encoding='utf-8') as file:
       map_results_mysql = json.load(file)
   with open('map_results_bm25.json', 'r', encoding='utf-8') as file:
       map_results_bm25 = json.load(file)
   with open('map_results_tfidf.json', 'r', encoding='utf-8') as file:
       map_results_tfidf = json.load(file)
except FileNotFoundError as e:
    print(f"Erro ao abrir o arquivo: {e}")

In [ ]:
map_qrels

In [ ]:
map_results_mysql

In [ ]:
map_results_bm25

In [ ]:
map_results_tfidf

Declarando funções de avaliação

In [ ]:
def preparar_relevancias(qrels_dict):
    """Transforma o dicionário de qrels em um dicionário de sets para busca rápida."""
    return {str(query_id): set(map(str, docs)) for query_id, docs in qrels_dict.items()}

In [ ]:
def compute_metrics(retrieved, relevant):
    retrieved_set = set(retrieved)
    relevant_set = set(relevant)
    tp = len(retrieved_set & relevant_set)
    precision = tp / len(retrieved) if retrieved else 0.0
    recall = tp / len(relevant) if relevant else 0.0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0
    return precision, recall, f1

In [ ]:
def calcular_metricas_macro(resultados_dict, relevancias_dict, limite_top_k=100):
    precisions, recalls, f1s = [], [], []
    y_true_all, y_score_all = [], []
    for query_id, retrieved in resultados_dict.items():
        relevant = relevancias_dict.get(str(query_id), set())
        retrieved = list(map(str, retrieved[:limite_top_k]))
        precision, recall, f1 = compute_metrics(retrieved, relevant)
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
        # Para curva PR: score decrescente conforme ranking
        y_true = [1 if doc in relevant else 0 for doc in retrieved]
        y_score = np.linspace(1, 0, len(retrieved), endpoint=False) if retrieved else []
        y_true_all.extend(y_true)
        y_score_all.extend(y_score)
    media_p = np.mean(precisions) if precisions else 0.0
    media_r = np.mean(recalls) if recalls else 0.0
    media_f1 = np.mean(f1s) if f1s else 0.0
    return media_p, media_r, media_f1, y_true_all, y_score_all


In [ ]:
def calcular_curva_pr_11_pontos(y_true, y_scores):
    precs, recalls, _ = precision_recall_curve(y_true, y_scores)
    niveis_recall = np.linspace(0.05, 1.0, 11)
    precisao_interpolada = []
    for nivel in niveis_recall:
        precisoes = [p for p, r in zip(precs, recalls) if r >= nivel]
        precisao_interpolada.append(max(precisoes) if precisoes else 0.0)
    return niveis_recall, precisao_interpolada


In [ ]:
relevancias = preparar_relevancias(map_qrels)


In [ ]:
relevancias

In [ ]:
# Converter valores do map_results_bm25 para integer
map_results_bm25 = {k: [int(doc) for doc in v] for k, v in map_results_bm25.items()}
map_results_tfidf = {k: [int(doc) for doc in v] for k, v in map_results_tfidf.items()}

In [ ]:
resultados_modelos = {
    "BM25": map_results_bm25,
    "MySQL": map_results_mysql,
    "TF-IDF": map_results_tfidf
}

metricas_modelos = {}
curvas_modelos = {}

In [ ]:
resultados_modelos

In [ ]:
for nome, resultados in resultados_modelos.items():
    media_p, media_r, media_f1, y_true_all, y_scores = calcular_metricas_macro(resultados, relevancias)
    metricas_modelos[nome] = (media_p, media_r, media_f1)
    curvas_modelos[nome] = calcular_curva_pr_11_pontos(y_true_all, y_scores)



In [ ]:
# Imprimir resultados das metricas
for nome, (precisao, recall, f1) in metricas_modelos.items():
    print(f"{nome} - Precisão: {precisao:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

In [ ]:
plt.figure(figsize=(7, 4))
for nome, (recall_pts, precisao_interp) in curvas_modelos.items():
    plt.plot(recall_pts, precisao_interp, marker='o', label=nome)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision em 11 pontos de Recall')
plt.ylim(0, 1)
plt.xlim(0, 1)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(7, 4))
for nome, (recall_pts, _) in curvas_modelos.items():
    plt.plot(recall_pts, recall_pts, marker='o', label=nome)
plt.xlabel('Recall')
plt.ylabel('Recall')
plt.title('Recall em 11 pontos')
plt.ylim(0, 1)
plt.xlim(0, 1)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(7, 4))
for nome, (recall_pts, precisao_interp) in curvas_modelos.items():
    f1_interp = [2*p*r/(p+r) if (p+r)>0 else 0 for p, r in zip(precisao_interp, recall_pts)]
    plt.plot(recall_pts, f1_interp, marker='o', label=nome)
plt.xlabel('Recall')
plt.ylabel('F1 Score')
plt.title('F1 Score em 11 pontos de Recall')
plt.ylim(0, 1)
plt.xlim(0, 1)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()
